# Benchmarking skforecast

Benchmarking is an essential part of the development process of **skforecast**. It provides a transparent view of how the **performance of the library** evolves across versions and helps users make informed decisions when choosing the right forecaster or configuration for their use case.

In this section, we present benchmark results that measure the execution time of key methods (`fit`, `predict`, etc.) and their **performance evolution across versions**, allowing the detection of improvements or regressions.

## Methodology

The benchmarking results presented here are generated using a **custom benchmarking script** located in the `benchmarks/` directory of the repository. This script executes a series of performance tests for the main forecasting classes and their methods, recording both execution time and variability across multiple runs.

To ensure **consistency and reproducibility**, all benchmarks are automatically executed as part of the continuous integration (CI) pipeline using GitHub Actions. This guarantees that each new release of the library is tested under the **same environment and dependency configuration**, making performance results directly comparable across versions.

Users who wish to reproduce the benchmarks locally can execute the same script by running:

```bash
# From skforecast root directory
python benchmarks/run_benchmarks.py
```

Results are stored in a structured format in `benchmarks/benchmarks.joblib`.

In [1]:
# Libraries
# ==============================================================================
import platform
import psutil
import numpy as np
import pandas as pd
import joblib

import sklearn
import skforecast
from benchmarks.utils import plot_benchmark_results

# Display all columns in pandas
pd.set_option("display.max_columns", None)

In [2]:
# Environment information
# ==============================================================================
print(f"Python version           : {platform.python_version()}")
print(f"skforecast version       : {skforecast.__version__}")
print(f"numpy version            : {np.__version__}")
print(f"pandas version           : {pd.__version__}")
print(f"scikit-learn version     : {sklearn.__version__}")
print(f"Computer network name    : {platform.node()}")
print(f"Processor type           : {platform.processor()}")
print(f"Platform type            : {platform.platform()}")
print(f"Number of physical cores : {psutil.cpu_count(logical=False)}")
print(f"Number of logical cores  : {psutil.cpu_count(logical=True)}")
print(f"Memory total             : {round(psutil.virtual_memory().total / 1e9, 2)} GB")

Python version           : 3.12.11
skforecast version       : 0.17.0
numpy version            : 2.1.3
pandas version           : 2.3.1
scikit-learn version     : 1.6.1
Computer network name    : ITES015-NB0029
Processor type           : Intel64 Family 6 Model 141 Stepping 1, GenuineIntel
Platform type            : Windows-11-10.0.26100-SP0
Number of physical cores : 8
Number of logical cores  : 16
Memory total             : 34.07 GB


In [3]:
import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="'force_all_finite' was renamed to 'ensure_all_finite'"
)

## Global results

In [4]:
# Load benchmark results
# ==============================================================================
results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark_all.head(2)

,forecaster_name,regressor_name,function_name,function_hash,method_name,run_time_avg,run_time_median,run_time_p95,run_time_std,n_repeats,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
0,ForecasterRecursive,DummyRegressor,ForecasterRecursive__create_train_X_y,59b823f1ff395872fac4f7578bd859fa,ForecasterRecursive__create_train_X_y,0.004261,0.004189,0.004373,0.000321,30,2025-08-25 11:25:01.633692,3.12.11,0.17.0,2.1.3,2.3.2,1.6.1,4.6.0,Linux-6.11.0-1018-azure-x86_64-with-glibc2.39,x86_64,4,16.77
1,ForecasterRecursive,DummyRegressor,ForecasterRecursive_fit,9d73eaf5faa980194d715362601eed68,ForecasterRecursive_fit,0.005308,0.005252,0.005579,0.000161,10,2025-08-25 11:25:01.691309,3.12.11,0.17.0,2.1.3,2.3.2,1.6.1,4.6.0,Linux-6.11.0-1018-azure-x86_64-with-glibc2.39,x86_64,4,16.77


## ForecasterRecursive

The [`ForecasterRecursive` class](https://skforecast.org/latest/user_guides/autoregresive-forecaster.html) is benchmarked under a fixed experimental setup to ensure fair comparison across library versions. Key conditions are:

| Condition          | Value                                                                                |
|:-------------------|:-------------------------------------------------------------------------------------|
| Regressor          | `sklearn.dummy.DummyRegressor` (to isolate forecaster overhead)                      |
| Dataset length     | 2000 synthetic observations (generated with `bench_forecaster_recursive._make_data`) |
| Exogenous features | 3                                                                                    |
| Prediction horizon | 100 steps ahead                                                                      |
| Backtesting split  | 1200 training, remaining for testing, step size = 50, no re-fitting (see [guide](https://skforecast.org/latest/user_guides/backtesting.html)) |

**Note:** In versions `< 0.14.0`, `ForecasterRecursive` was named `ForecasterAutoreg`.  

In [5]:
# Plot benchmark results
# ==============================================================================
plot_benchmark_results(
    results_benchmark_all,
    forecaster_names = ['ForecasterRecursive', 'ForecasterAutoreg'],
    regressors       = ['DummyRegressor'],
    add_mean         = True,
    add_median       = True
)